In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from pathlib import Path
pd.set_option('display.max_columns', 500)

In [3]:
cwd = Path(os.getcwd())
base_wd = cwd.parent.parent 
data_path = base_wd / Path("data/parquet_files/train")

In [4]:
df_base = pd.read_parquet(data_path / "train_base.parquet")
df_base.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [5]:
df_credit = pd.read_parquet(data_path / "train_credit_bureau_a_1_0.parquet")

In [6]:
df_credit[df_credit['case_id'] == 388]

,case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,lastupdate_1112D,lastupdate_388D,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax2date_1002D,overdueamountmax2date_1142D,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
0,388,NaN,NaN,4408ff0f,a55475b1,7241344e,a55475b1,NaN,NaN,NaN,2023-06-20,None,None,2018-06-20,None,374419.5,0.0,a55475b1,0.0,NaN,NaN,7.0,2018.0,NaN,a55475b1,55b002a9,NaN,NaN,NaN,2019-01-24,None,7811.4463,NaN,NaN,NaN,0,2.0,1.0,NaN,60.0,NaN,54.0,0.0,NaN,None,None,0.0,NaN,NaN,260093.7,NaN,0.0,0.0,NaN,None,None,0.0,NaN,NaN,7.0,2018.0,NaN,NaN,30.0,NaN,NaN,96a8fdfe,a55475b1,None,NaN,NaN,ab3c25cf,ab3c25cf,NaN,268897.62,0.0,0.0,374419.5,0.0
1,388,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,135806.0,2020-08-06,None,None,2018-08-06,None,NaN,NaN,a55475b1,0.0,NaN,NaN,8.0,2018.0,NaN,a55475b1,P204_66_73,8742.8,NaN,NaN,2019-01-11,None,8742.8000,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,None,None,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,None,None,0.0,NaN,NaN,8.0,2018.0,NaN,NaN,NaN,NaN,NaN,60c73645,a55475b1,None,NaN,114325.805,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
2,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
3,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
4,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
5,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,

In [7]:
df_base_credit = df_base.merge(df_credit, on = 'case_id')

In [8]:
df_base_credit = df_base_credit[ (df_base_credit['dateofrealrepmt_138D'] < df_base_credit['date_decision'])]

In [9]:
mean_cols = ['numberofcontrsvalue_358L', 'nominalrate_281L']

In [10]:
final_df = pd.DataFrame()
for col in mean_cols:
    final_df[f'mean_{col}'] = df_base_credit[df_base_credit['target'] == 1].groupby('case_id')[col].mean()



In [11]:
result_df = df_base_credit.groupby('case_id').agg(mean_numberofcontrsvalue_358L=('numberofcontrsvalue_358L', 'mean'),
                                                  mean_nominalrate_281L=('nominalrate_281L', 'mean'))


In [12]:
from omegaconf import OmegaConf
import pandas as pd
config = OmegaConf.load("../../config.yaml")


def create_aggration_dataframe(cfg: dict, df: pd.DataFrame, itteration: int) -> pd.DataFrame:
    df = df[df[cfg.time_col[0]] < df['date_decision']]
    aggregation_specs = {}
    for column in cfg.agg_columns:
        aggregation_specs[f'{column.name}_{column.aggregation}_{itteration}'] = (column.name , column.aggregation)
    return df.groupby('case_id').agg(**aggregation_specs)

In [13]:
from pathlib import Path
import os
cwd = Path(os.getcwd())
base_wd = cwd.parent.parent 

def get_orderd_data_files(data_path: Path, all_file_sources: list[str]) -> list[dict]:
    file_dict = []
    all_files =  os.listdir(data_path)
    for source in all_file_sources:
    #     file_dict.append({source: [x if source in x else None for x in all_files].dropna()})

        filtered_files = filter(lambda x: source in x, all_files)
        filtered_files = list(filter(None, filtered_files))
        file_dict.append({source: filtered_files})
        
        
    return file_dict

        

In [14]:
# all_file_sources = list(config.data.keys())
# data_structure = get_orderd_data_files(base_wd / "data/parquet_files/train", all_file_sources = all_file_sources)
# for source_file in data_structure:

#     source_name = list(source_file.keys())[0]
#     files = list(source_file.values())[0]
#     cfg_columns = config.data[source_name]
#     df_base = pd.read_parquet(base_wd / "data/parquet_files/train/train_base.parquet")
#     df  = pd.DataFrame(df_base['case_id'])
#     for i, file in enumerate(files):
#         print(file)
#         df_file = pd.read_parquet(base_wd / f"data/parquet_files/train/{file}")
#         df_file = df_file[['case_id'] +[ col.name for col in cfg_columns.agg_columns] + [cfg_columns.time_col[0]]]
#         df_combined = df_base.merge(df_file, on = 'case_id', how = 'left')
#         df_agg = create_aggration_dataframe(cfg_columns, df_combined, i)
 
#         df = df.merge(df_agg, on = 'case_id', how ='left')

# df.head()

In [20]:
df = pd.read_feather(data_path / "processed_train.feather")
df.head()

,case_id,numberofcontrsvalue_358L_max_0,nominalrate_281L_mean_0,numberofcontrsvalue_358L_max_1,nominalrate_281L_mean_1,numberofcontrsvalue_358L_max_2,nominalrate_281L_mean_2,numberofcontrsvalue_358L_max_3,nominalrate_281L_mean_3,last180dayaveragebalance_704A_last_0
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df[df['last180dayaveragebalance_704A_last_0'].notnull()]

,case_id,numberofcontrsvalue_358L_max_0,nominalrate_281L_mean_0,numberofcontrsvalue_358L_max_1,nominalrate_281L_mean_1,numberofcontrsvalue_358L_max_2,nominalrate_281L_mean_2,numberofcontrsvalue_358L_max_3,nominalrate_281L_mean_3,last180dayaveragebalance_704A_last_0
58124,103558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
58393,103827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
58822,104256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
58860,104294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
59160,104594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
1525550,2702305,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,0.0
1525782,2702537,NaN,NaN,NaN,NaN,NaN,NaN,13.0,42.00,0.0
1525930,2702685,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,0.0
1525994,2702750,NaN,NaN,NaN,NaN,NaN,NaN,9.0,41.75,0.0


In [16]:
import pandas as pd
pd.read_parquet(data_path / "train_credit_bureau_a_2_0.parquet")

,case_id,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,subjectroles_name_541M,subjectroles_name_838M
0,388,8fd95e4b,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,NaN,NaN,2.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,ab3c25cf
1,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,1,NaN,NaN,3.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
2,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,2,NaN,NaN,4.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
3,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,3,NaN,NaN,5.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
4,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,4,NaN,NaN,6.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296026,2548729,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,2,31,NaN,NaN,9.0,NaN,NaN,NaN,2019.0,NaN,a55475b1,a55475b1
5296027,2548729,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,2,32,NaN,NaN,10.0,NaN,NaN,NaN,2019.0,NaN,a55475b1,a55475b1
5296028,2548729,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,2,33,NaN,NaN,11.0,NaN,NaN,NaN,2019.0,NaN,a55475b1,a55475b1
5296029,2548729,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,2,34,NaN,NaN,12.0,NaN,NaN,NaN,2019.0,NaN,a55475b1,a55475b1
